In [1]:
# Install packages
!pip install -U accelerate
!pip install -U transformers
!pip install datasets
!pip install torch
!pip install sentencepiece
!pip install numpy
# !pip install google-cloud-storage
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00


In [2]:
# Imports
import torch
import transformers
import numpy as np
import pandas as pd

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)

In [3]:
# Model
device = "cuda"
# device = "cpu"
model_name = 'adenhaus/mt5-base-tata-trans-blueprints'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/941 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
def split_verbalisation(text):
  split_text = text.split("Verbalisation: ")

  if len(split_text) > 1:
    return split_text[1]
  else:
    return text

In [12]:
# Predict function
def generate_verbalisation(model, tokenizer, example):
    # source = example['linearized_input']
    # target = example['target']
    input_ids = tokenizer(example)["input_ids"]
    # input_ids = torch.LongTensor(input_ids).view(1, -1).to(model.device)
    input_ids = torch.LongTensor(input_ids).view(1, -1).to(device)
    generated_ids = model.generate(input_ids, max_new_tokens=500)
    prediction = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    prediction = split_verbalisation(prediction)

    print(prediction)

    return prediction

In [6]:
# Load test set
df = pd.read_csv("test.csv", sep='\t')

In [7]:
len(df)

763

In [8]:
df.iloc[35]['linearized_input']

'Attitudes toward Wife Beating | Percent of women and men age15-49 who believe that a husband is justified in beating his wife for the following reasons: | (Women, Wife burns the food, 14) (Men, Wife burns the food, 8) (Women, Wife argues with him, 21) (Men, Wife argues with him, 13) (Women, Wife goes out without telling him, 25) (Men, Wife goes out without telling him, 13) (Women, Wife neglects children, 25) (Men, Wife neglects children, 14) (Women, Wife refuses to have sex with him, 19) (Men, Wife refuses to have sex with him, 11) (Women, Any of these, 35) (Men, Any of these, 25)'

In [9]:
df.iloc[35]['target']

'More than one-third of women (35%) and one-quarter of men agree that a husband is justified in beating his wife for at least one of these reasons: if she burns the food, argues with him, goes out without telling him, neglects the children, or refuses to have sex with him.'

In [10]:
generate_verbalisation(model, tokenizer, df.iloc[35]['linearized_input'])

Only 13% of women think that wife beating is justified in beating his wife.


'Only 13% of women think that wife beating is justified in beating his wife.'

In [ ]:
# Generate output dataset for evaluation
out_df = pd.DataFrame(columns=['preds', 'refs'])
out_df['refs'] = df['target']
out_df['preds'] = df['linearized_input'].apply(lambda x: generate_verbalisation(model, tokenizer, x))
out_df.to_csv('mt5-base-preds.csv', sep='\t', index=False)

خمسة وعشرون في المئة من النساء في الحضر اللاتي يعتبرن جنسًا أو أكثر.
الرجال أكثر احتمالاً للزواج الأول في الوقت المناسب، مقارنةً بنسبة الرجال في الوقت المناسب.
يعتبر الرجال الذين يعتبرون جنسيين أو أكثر في الوقت المحدد للزواج الأول، مقارنةً بنسبة 54 % في الوقت المحدد للزواج الأول.
The percentage of urban households who have had a child or her baby is highest in urban areas (21,2 years versus 21 years for the percentage of urban households).
The percentage of men who have had a first sexual intercourse in the first month preceding the survey ranges from 254 in the first month preceding the survey to 254 in the first month preceding the survey.
The percentage of women who have had a first sexual intercourse in the first month preceding the survey ranges from 54% in the first month preceding the survey to 54% in the first month preceding the survey.
Blueprint: jeune. Les femmes urbaines ont un taux de plus bas de l’âge de 30 ans ou plus avant l’âge de 30 ans ou plus avant l’âge de 30 ans o

In [ ]:
# out_df = out_df.loc[~out_df['preds'].str.contains('Blueprint: ')]
# out_df.to_csv('mt5-base-preds.csv', sep='\t', index=False)